
# RAG PDF — Notebook d'essai **manuel** (sans Gradio)

Ce notebook :
1. Installe les dépendances Python requises  
2. Vérifie la présence des binaires système pour l’OCR (Poppler/Tesseract)  
3. Utilise vos **fonctions** dans le dossier `fonctions/` pour ingérer tous les PDF de `data/` (texte + tableaux + OCR)  
4. Persiste la base vectorielle dans `.chroma_notebook/`  
5. Permet des **tests manuels** (retrieval + LLM)  


> **Important** : Le dossier `fonctions/` doit contenir : `config.py`, `embeddings.py`, `ingestion.py`, `retrieval.py`, `utils.py`.


## 0) Préparer l'environnement & vérifier le PATH

In [1]:

import os, pathlib, sys

root = pathlib.Path(".").resolve()
print("Racine du projet:", root)

# S'assurer que le dossier du projet est bien dans le PYTHONPATH
if str(root) not in sys.path:
    sys.path.append(str(root))

print("\nContenu de la racine:")
for p in sorted(root.glob("*")):
    print("-", p.name)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # ou "true" si tu veux le laisser activé


Racine du projet: /Users/natachanjongwayepnga/Downloads/GENAI_PRO_Cohorte_1/Semaine5/01_RAG_GPT

Contenu de la racine:
- .DS_Store
- .env
- RAG_PDF_notebook.ipynb
- README.md
- app.py
- constraints.txt
- data
- fonctions
- requirements.txt


## 1) Installer les dépendances Python (exécuter une seule fois)

In [2]:
#pip install -r requirements.txt -c constraints.txt

## 2) Vérifier les binaires système pour l’OCR (Poppler / Tesseract)

In [3]:

import shutil

def check_bin(name):
    path = shutil.which(name)
    print(f"{name}: {'OK → ' + path if path else 'NON TROUVÉ'}")
    return path

poppler = check_bin("pdftoppm")
tess = check_bin("tesseract")

if not poppler:
    print("\n⚠️ Poppler manquant. Installez-le sur votre machine :")
    print("- macOS:  brew install poppler")
    print("- Ubuntu: sudo apt-get install poppler-utils")

if not tess:
    print("\n⚠️ Tesseract manquant. Installez-le sur votre machine :")
    print("- macOS:  brew install tesseract")
    print("- Ubuntu: sudo apt-get install tesseract-ocr")


pdftoppm: OK → /opt/homebrew/bin/pdftoppm
tesseract: OK → /opt/homebrew/bin/tesseract


## 3) Paramètres/Environnement (optionnel)

In [4]:

import os
from pathlib import Path

# (Optionnel) Clé OpenAI pour utiliser un LLM OpenAI dans le notebook
# Si vous laissez vide, la cellule LLM échouera — c'est normal ; vous pouvez tester uniquement le retrieval.
os.environ.setdefault("OPENAI_API_KEY", "")  # <-- Renseignez votre clé si vous souhaitez tester le LLM ici

print("DATA_DIR attendu :", Path("data").resolve())
print("Vector store (notebook) :", Path(".chroma").resolve())


DATA_DIR attendu : /Users/natachanjongwayepnga/Downloads/GENAI_PRO_Cohorte_1/Semaine5/01_RAG_GPT/data
Vector store (notebook) : /Users/natachanjongwayepnga/Downloads/GENAI_PRO_Cohorte_1/Semaine5/01_RAG_GPT/.chroma


## 4) Importer les fonctions depuis `fonctions/`

In [5]:

from fonctions.config import DATA_DIR, PERSIST_DIR, COLLECTION_NAME, TOP_K, ENABLE_OCR, OCR_MIN_TEXT_CHARS
from fonctions.ingestion import ingest_all_pdfs
from fonctions.retrieval import get_vectorstore, retrieve_docs

print("Config:")
print("- DATA_DIR        :", DATA_DIR)
print("- PERSIST_DIR     :", PERSIST_DIR)
print("- COLLECTION_NAME :", COLLECTION_NAME)
print("- TOP_K           :", TOP_K)
print("- ENABLE_OCR      :", ENABLE_OCR)
print("- OCR_MIN_TEXT_CHARS:", OCR_MIN_TEXT_CHARS)


Config:
- DATA_DIR        : /Users/natachanjongwayepnga/Downloads/GENAI_PRO_Cohorte_1/Semaine5/01_RAG_GPT/data
- PERSIST_DIR     : .chroma
- COLLECTION_NAME : pdf_collection
- TOP_K           : 8
- ENABLE_OCR      : True
- OCR_MIN_TEXT_CHARS: 120


## 5) Ingestion de tous les PDFs (`data/*.pdf`) & persistance dans `.chroma_notebook/`

In [6]:

#  Placez vos PDF dans le dossier data/ avant d’exécuter cette cellule
vs = ingest_all_pdfs()
print("✅ Ingestion terminée et base vectorielle persistée dans", PERSIST_DIR)


✅ Ingestion terminée et base vectorielle persistée dans .chroma


## 6) (Re)charger la base persistée (si redémarrage du kernel)

In [7]:

vs = get_vectorstore()
print("Vector store chargé :", vs is not None)


Vector store chargé : True


## 7) RAG minimal (retrieval MMR + LLM). **Note** : nécessite `OPENAI_API_KEY` si vous voulez poser des questions au LLM.

In [8]:

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def _normalize_score(s: float) -> float:
    if s < 0:      # cosine [-1,1]
        s = (s + 1.0) / 2.0
    elif s > 1:    # distance
        s = 1.0 / (1.0 + s)
    return max(0.0, min(1.0, s))

def _load_llm():
    key = os.getenv("OPENAI_API_KEY", "")
    if not key:
        raise RuntimeError("OPENAI_API_KEY manquante. Renseignez-la pour tester le LLM dans ce notebook.")
    return ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

PROMPT = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant factuel. Réponds en français UNIQUEMENT avec le contexte fourni. Cite les pages: [p. X]."),
    ("user", """Question:
{question}

Contexte:
{context}

Règles:
- Sois concis et précis
- Termine par "Sources: ..." avec les pages
""")
])
PARSER = StrOutputParser()
LLM = None

def _format_context(hits):
    def fmt_piece(doc):
        txt = doc.page_content or ""
        typ = (doc.metadata or {}).get("type", "text")
        # pas de troncature pour les tables/table_flat
        if typ not in ("table", "table_flat") and len(txt) > 1600:
            txt = txt[:1600] + " ..."
        return f"(p. {doc.metadata.get('page')}) {txt}"
    parts = [fmt_piece(d) for d, _ in hits]
    pages = sorted({(d.metadata or {}).get("page") for d, _ in hits if (d.metadata or {}).get("page") is not None})
    return "\n\n".join(parts), pages

def ask(question: str, k: int = 8):
    try:
        raw = vs.max_marginal_relevance_search_with_score(
            question, k=k, fetch_k=max(40, k*8), lambda_mult=0.1
        )
    except Exception:
        raw = vs.similarity_search_with_relevance_scores(question, k=k)
    hits = [(doc, _normalize_score(score)) for doc, score in raw]
    if not hits:
        return "Aucun passage pertinent trouvé.", []
    context, pages = _format_context(hits)
    global LLM
    if LLM is None:
        LLM = _load_llm()
    answer = (PROMPT | LLM | PARSER).invoke({"question": question, "context": context})
    if pages:
        answer += "\n\nSources: " + ", ".join([f"p. {p}" for p in pages])
    return answer, hits


## 8) Essai manuel — posez une question au LLM (contexte = vos PDFs ingérés)

In [9]:

question = "Quel est le total revenue (chiffre d’affaires) de Tesla au T1 2023 ?"
reponse, hits = ask(question, k=8)
print(reponse)


Le chiffre d'affaires total de Tesla au T1 2023 s'élève à 23,3 milliards de dollars, ce qui représente une croissance de 24 % par rapport à l'année précédente [p. 4]. 

Sources: [p. 4], [p. 22].

Sources: p. 2, p. 4, p. 5, p. 8, p. 18, p. 19, p. 22, p. 27


## 9)  Inspecter les passages renvoyés par le retrieval (sans LLM)

In [10]:

question = "Combien de véhicules ont été livrés au T1 2023 ?"
topk = 10
pairs = retrieve_docs(vs, question, k=topk)
for d, sc in pairs[:5]:
    typ = (d.metadata or {}).get("type")
    pg = (d.metadata or {}).get("page")
    preview = (d.page_content or "").replace("\n", " ")[:240]
    print(f"[p.{pg}] score={sc:.3f} type={typ} :: {preview} …")


[p.6] score=0.314 type=text :: Source: Tesla estimates based on ACEA; Autonews.com; CAAM – light-duty vehicles only TTM = Trailing twelve months7 In Q1, we produced a record number of vehicles, thanks to ongoing ramps at our  factories in Austin and Berlin. We remain com …
[p.5] score=0.295 type=text :: Q1-2022 Q2-2022 Q3-2022 Q4-2022 Q1-2023 YoY Model S/X production 14,218 16,411 19,935 20,613 19,437 37% Model 3/Y production 291,189 242,169 345,988 419,088 421,371 45% Total production 305,407 258,580 365,923 439,701 440,808 44% Model S/X  …
[p.18] score=0.260 type=text :: 0 1 2 3 4 5 6 2Q-2020 3Q-2020 4Q-2020 1Q-2021 2Q-2021 3Q-2021 4Q-2021 1Q-2022 2Q-2022 3Q-2022 4Q-2022 1Q-2023 0 1 2 3 4 5 6 2Q-2020 3Q-2020 4Q-2020 1Q-2021 2Q-2021 3Q-2021 4Q-2021 1Q-2022 2Q-2022 3Q-2022 4Q-2022 1Q-2023 0.0 0.1 0.2 0.3 0.4  …
[p.19] score=0.246 type=text :: 0 2 4 6 8 10 12 14 16 18 20 2Q-2020 3Q-2020 4Q-2020 1Q-2021 2Q-2021 3Q-2021 4Q-2021 1Q-2022 2Q-2022 3Q-2022 4Q-2022 1Q-2023 0 2 4 6 8 10 12 14 16